In [1]:
import cv2
import numpy as np
import time
from jetbot import ObjectDetector
from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [2]:
from jetbot import Camera
camera = Camera.instance(width=300, height=300)

In [13]:
detections = model(camera.value)
print(detections)

[[{'label': 64, 'confidence': 0.6675812602043152, 'bbox': [0.32334965467453003, 0.3238368630409241, 0.5677865147590637, 0.7766782641410828]}, {'label': 86, 'confidence': 0.3881540596485138, 'bbox': [0.35922643542289734, 0.49570202827453613, 0.5629936456680298, 0.7788211107254028]}]]


In [14]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
target_obj_tall = 1
target_obj_short = 1
obstacle_obj_1 = 64 #cup
obstacle_obj_2 = 86 #plant
Standard_Motor_speed = 0.35
object_info_dict = {1:("tall_object", 64), 86:("short_object", 74), 44:("Apple", 30), 64:("Banana", 40)}
image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)
robot = Robot()

In [15]:
def leftTurn(incremental,delay):
    robot.left(incremental) # arbitrary small value
    time.sleep(delay) # can change, might want to go higher
    robot.stop()
    time.sleep(1.5*delay)
    robot.forward(Standard_Motor_speed)
    time.sleep(1.5*delay)
    robot.stop()
    time.sleep(1)
    robot.right(incremental)
    time.sleep(delay)
    robot.stop()
    time.sleep(delay)
    robot.forward(Standard_Motor_speed)
    
def rightTurn(incremental, delay):
    robot.right(incremental) # arbitrary small value
    time.sleep(delay) # can change, might want to go higher
    robot.stop()
    time.sleep(1.5*delay)
    robot.forward(Standard_Motor_speed)
    time.sleep(1.5*delay)
    robot.stop()
    time.sleep(1)
    robot.left(incremental) # arbitrary small value
    time.sleep(delay)
    robot.stop()
    time.sleep(delay)
    robot.forward(Standard_Motor_speed)
    
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / (2.0 - 0.5)
    center_y = (bbox[1] + bbox[3]) / (2.0 - 0.5)
    area = ((bbox[2] - bbox[0])*width)*((bbox[3] - bbox[1])*height)
    return (center_x*width, center_y*height,area)

def steer(filterred_obstacle_detection):
    obstacle_centriod = []
    obstacle_area = []
    obstacle_label = []
    center_frame =  width/2.0
    print("Center frame is " + str(center_frame))
    for det in filterred_obstacle_detection:
        center_x, center_y, area = detection_center(det)
        obstacle_centriod.append(center_x)
        print(det['label'])
        print("Center_x " +str(center_x))
        print("Center_y " + str(center_y))
    obstacle_list_len = len(obstacle_centriod)
    

    if obstacle_list_len == 2:
        center_x_0 = obstacle_centriod[0]
        center_x_1 = obstacle_centriod[1]
        if (center_x_0 < center_frame) and (center_x_1 < center_frame):
            print("Right turn")
            rightTurn(0.25, 0.3)
        elif (center_x_0 > center_frame) and (center_x_1 > center_frame):
            print("Left turn")
            leftTurn(0.25,0.3)
        else:
            print("take a long turn")
            leftTurn(0.25, 0.5)

        print("Found 2 obstacles in the frame")
        
    elif obstacle_list_len == 1:
        center_x = obstacle_centriod[0]
        print("Found 1 obstacles in the frame")
        if center_x < center_frame:
            print("Turning right")
            rightTurn(0.3, 0.3)
        else:
            print("Turning Left")
            leftTurn(0.3, 0.3) 
        print("Found 1 obstacles in the frame")
    else:
        return
def estimate_distance(detection):
    focal_length = 105.0  #need to caliborate your camera to get this
    obj_info = object_info_dict[detection['label']]
    bbox = detection['bbox']  
    obj_width = obj_info[1]
    obj_width_in_img = width * (bbox[2] - bbox[0])        
    dist = obj_width * focal_length / obj_width_in_img
    return dist


In [18]:
robot.stop()

In [16]:
def execute(change):
    
    image = change['new']
    #gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #blur = cv2.GaussianBlur(gray,(5,5),cv2.BORDER_DEFAULT)
    # compute all detected objects
    detections = model(image)
    is_local_steering_in_progress =0
    filtered_detections = []
    filterred_obstacle_detection = []
    print("Found " + str(len(detections[0])) + " objects")
    # visualized target and obstacle objects
    for det in detections[0]:
        bbox = det['bbox']
        label = det['label']
        print(label)
        if(label == target_obj_tall or label == target_obj_short):
            dist = estimate_distance(det)
            filtered_detections.append(det)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            cv2.putText(image, object_info_dict[label][0] + ":" + "{:.2f}".format(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255),2)
        elif(label == obstacle_obj_1 or label == obstacle_obj_2):
            dist = estimate_distance(det)
            is_local_steering_in_progress = 1;
            filtered_detections.append(det)
            filterred_obstacle_detection.append(det)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
            cv2.putText(image, object_info_dict[label][0] + ":" + "{:.2f}".format(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),2)
        else:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            cv2.putText(image, str(label), (int(width * bbox[0]), int(height * bbox[1]) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0),2)
       
    
    # update image widget
    #image_widget.value = bgr8_to_jpeg(image)
    
    #if(is_target_reached(filtered_detections)):
    #    camera.unobserve_all()
     #   robot.stop()
     #   return
    
    # if in local steering mode, return
    if(is_local_steering_in_progress):
        steer(filterred_obstacle_detection)
        is_local_steering_in_progress = 0
        #robot.stop()
        print("is_local_steering_in_progress")
        return
    
    
    #if(is_local_steering_in_progress == False):
     #   lock_and_move_to_target(filtered_detections)
    


In [37]:
image = widgets.Image(format='jpeg', width=400, height=400)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [27]:
execute({'new': camera.value})

Found 2 objects
64
64
Center frame is 150.0
64
Center_x 142.6639199256897
Center_y 195.04966139793396
64
Center_x 142.3884630203247
Center_y 204.49676513671875
Right turn
Found 2 obstacles in the frame
is_local_steering_in_progress


In [28]:
robot.stop()